In [68]:
import sys, os, glob, random, copy, time, shutil, re
import itertools
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
# random.seed(666)
from collections import Counter
import numpy as np
from itertools import cycle
import pandas as pd
import seaborn as sns
from matplotlib import pyplot
import jellyfish
try:
    def hamming_distance(s1, s2):
        if s1 == s2:
            return 0
        else:
            return jellyfish.hamming_distance(s1, s2)
    assert(hamming_distance('ABC', 'ABCD') == 1)
except:
    def hamming_distance(s1, s2):
        if s1 == s2:
            return 0
        else:
            return jellyfish.hamming_distance(unicode(s1), unicode(s2))
    assert(hamming_distance('ABC', 'ABCD') == 1)

import traceback
class nostdout(object):
    def __enter__(self):
        self.stdout = sys.stdout
        sys.stdout = self
    def __exit__(self, type, value, traceback):
        sys.stdout = self.stdout
        if type is not None:
            raise
    def write(self, x): pass

In [69]:
def plot_AAseq_CDF(df, verbose=True):
    plot_dir = 'plots/AAseq_CDF'
    if not os.path.exists(plot_dir):
        os.makedirs(plot_dir)

    def AUC(res, locus, st, cut):
        try:
            counts = Counter(res[locus][st])
            hd = [k for k, v in counts.items() if k <= cut]
            c = [v for k, v in counts.items() if k <= cut]
            hd, c = zip(*sorted(zip(hd, c)))
            acc = [sum(c[0:i]) + c[i] for i in range(len(c))]
            sc = sum(v for k, v in counts.items())
            auc = sum(acc[i]*(hd[i+1]-hd[i]) for i in range(len(hd)-1)) / sc
            return(auc)
        except:
            return(0)
    

    def CDF_plot(res, xlimit, pnam):
        # %matplotlib inline
        from matplotlib.backends.backend_pdf import PdfPages
        pp = PdfPages(pnam)
        for locus in res:
            fig, ax = pyplot.subplots(figsize=(14,10))
            for st in res[locus]:
                if st[0][0:3] == '3FT' and st[1][0:3] == '3FT':
                    color = 'green'
                elif st[0][0:3] == 'PLA' and st[1][0:3] == 'PLA':
                    color = 'blue'
                else:
                    color = 'red'
                auc = AUC(res, locus, st, xlimit)
                ax = sns.distplot(
                    res[locus][st],
                    label='{} vs. {} AUC {:.1f}'.format(st[0], st[1], auc),
                    color=color,
                    kde=False,
                    bins=list(range(0, 1000)),
                    norm_hist=True,  # On/off to normalize y-axis
                    hist_kws={'histtype':'step', 'cumulative':True, 'lw':3}
                )
                lgd = ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
                ax.set_title('Locus: {}'.format(locus))
                ax.set_xlim(0, xlimit)
            fig.savefig(pp, format='pdf', bbox_extra_artists=(lgd,), bbox_inches='tight')
        pp.close()


    def compare_AAseq(table, trim=0, downsample=False, weighted=False, mode='naiveAA'):
        hamming_dist_xlimit = 10 if mode == 'Fcdr3' else 30
        df = table.copy(deep=True)
        # Trim out small clonal families:
        df = df[df['Nseqs'] > trim]

        row_part = dict()
        dist_table = dict()

        samples = list(set(df['sample']))
        loci = set(df['locus'])

        if downsample:
            for locus in loci:
####
#                if locus != 'IgH':
#                    continue
                smallest = 9999999999
                for sample in samples:
                    print(len(df[(df['locus'] == locus) & (df['sample'] == sample)]))
                    size = sum(np.array(df['locus'] == locus) & np.array(df['sample'] == sample))
                    if size < smallest:
                        smallest = size
                print('Downsampling to:', smallest)
                for sample in samples:
                    # Finding the downsample:
                    ds = df[(df['locus'] == locus) & (df['sample'] == sample)].sample(smallest, replace=False)
                    # Dropping all columns:
                    df = df[np.invert(np.array(((df['locus'] == locus) & (df['sample'] == sample))))]
                    # Adding the downsample:
                    df = df.append(ds)
        v_grps = set(df['v_grp'])
        d_grps = set(df['d_grp'])
        j_grps = set(df['j_grp'])
        vdj_len = set(df['vdj_len'])


        for index, row in df.iterrows():
            # kt = (row['locus'], row['v_grp'], row['d_grp'], row['j_grp'], row['vdj_len'])
            kt = (row['locus'], row['v_grp'], row['d_grp'], row['j_grp'])
            if kt not in row_part:
                row_part[kt] = [index]
            else:
                row_part[kt].append(index)
        sumstat = [len(l) for l in row_part.values()]
        #    print('These are the number of entries in each partition:', list(map(str, sumstat)))

        # res[locus][s1:s2] = [dist, 5, 2, 0,...]
        res = dict()
        for locus in loci:
####
#            if locus != 'IgH':
#                continue
            res[locus] = dict()
            for i in range(len(samples)):
                for j in range(i+1, len(samples)):
                    si = samples[i]
                    sj = samples[j]
                    res[locus][(si, sj)] = list()
                    for kt, li in row_part.items():
                        if locus != kt[0]:
                            continue
                        if mode == 'naiveAA':
                            seqi = list(df.loc[li][df.loc[li]['sample'] == si]['naiveAA'])
                            seqj = list(df.loc[li][df.loc[li]['sample'] == sj]['naiveAA'])
                        elif mode == 'Fseq':
                            seqi = [Counter(i.split(':')).most_common()[0][0] for i in df.loc[li][df.loc[li]['sample'] == si]['input_seqsAA']]
                            seqj = [Counter(i.split(':')).most_common()[0][0] for i in df.loc[li][df.loc[li]['sample'] == sj]['input_seqsAA']]
                        elif mode == 'Fcdr3':
                            seqi = [Counter([str(Seq(s[int(b):int(e)], generic_dna).translate()) for s in i.split(':')]).most_common()[0][0] for i, b, e in zip(df.loc[li][df.loc[li]['sample'] == si]['input_seqs'], df.loc[li][df.loc[li]['sample'] == si]['CDR3_start'], df.loc[li][df.loc[li]['sample'] == si]['CDR3_end'])]
                            seqj = [Counter([str(Seq(s[int(b):int(e)], generic_dna).translate()) for s in i.split(':')]).most_common()[0][0] for i, b, e in zip(df.loc[li][df.loc[li]['sample'] == sj]['input_seqs'], df.loc[li][df.loc[li]['sample'] == sj]['CDR3_start'], df.loc[li][df.loc[li]['sample'] == sj]['CDR3_end'])]
                        else:
                            raise BaseException('mode not supported')
                        if len(seqi) == 0 or len(seqj) == 0:
                            continue
                        hdj = [min(hamming_distance(ni, nj) for ni in seqi) for nj in seqj]
                        hdi = [min(hamming_distance(ni, nj) for nj in seqj) for ni in seqi]
                        if weighted is True:
                            cf_abui = list(df.loc[li][df.loc[li]['sample'] == si]['total_abundance'])
                            cf_abuj = list(df.loc[li][df.loc[li]['sample'] == sj]['total_abundance'])
                            hdj = [h for h, a in zip(hdj, cf_abuj) for i in range(a)]
                            hdi = [h for h, a in zip(hdi, cf_abui) for i in range(a)]
                        res[locus][(si, sj)].extend(hdi)
                        res[locus][(si, sj)].extend(hdj)

        ds = '_downsampled' if downsample else ''
        we = '_weighted' if weighted else ''
        pnam = plot_dir + '/{}_comparison_trim{}{}{}.pdf'.format(mode, trim, ds, we)
        CDF_plot(res, hamming_dist_xlimit, pnam)


    trim_list = [0, 5]
    downsample_list = [True, False]
    weighted_list = [True, False]
    mode = ['naiveAA', 'Fseq', 'Fcdr3']
    combs = list(itertools.product(*[trim_list, downsample_list, weighted_list, mode]))
    for c in combs:
        if verbose:
            compare_AAseq(df, *c)
        else:
            with nostdout():
                compare_AAseq(df, *c)

In [66]:
#df = pd.read_pickle('data/cf.pickle')
df = pd.read_pickle('data/cf_master.pickle')
#df = df.sample(n=1000)

In [71]:
### Placeholder for all the plotting functions:
plot_AAseq_CDF(df, verbose=False)